## Bibliotecas


In [1]:
%matplotlib inline
# importar biblioteca Pandas
import pandas as pd

# importar Matplotlib biblioteca de gráficos
import matplotlib.pyplot as plt

# importar Seaborn biblioteca de gráficos
import seaborn as sns

import geopandas as gpd

# Importar Numpy, train_test_split
import numpy as np
from sklearn.model_selection import train_test_split

# importa o modelo de regressão linear, metrics, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import metrics

from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import LabelEncoder

# acesso ao so e ao sys
import os
import sys

# Garbage collector, para recuperar memória após carregar o df inicial
import gc

import graphviz.backend as be
import statistics as sts

# Importar algoritmo
# from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from mpl_toolkits.mplot3d import axes3d

# import dtreeviz

random_state = 1234

# controlar as notificações
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.float_format", lambda x: "%.8f" % x)

%config InlineBackend.figure_format = 'retina' # Make visualizations look good
# %config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [64]:
url = "./data_input/faccionados/faccionados_analise_vinculo.csv"

In [65]:
df = pd.read_csv(url, encoding="utf8", low_memory=False)

In [66]:
print(list(df.columns))
len(df.columns)

['Unnamed: 0', 'nome_completo', 'vulgo_alcunha', 'faccao', 'faccao_funcao', 'bairro_atual', 'cidade_atual', 'uf_atual', 'area_atuacao']


9

Tornar todos os NaN como 0 = não informado


In [67]:
df.fillna("NI", inplace=True)

In [68]:
df.head(5)

,Unnamed: 0,nome_completo,vulgo_alcunha,faccao,faccao_funcao,bairro_atual,cidade_atual,uf_atual,area_atuacao
0,1,KAYO NASCIMENTO DE MAGALHÃES,CABRA/ 99,NI,FINANCEIRO,NI,NI,NI,NI
1,2,JOSÉ MARIANO DOS SANTOS,RATO,NI,BRAÇO DIREITO,NI,NI,NI,NI
2,3,GIVALDO BARBOSA DE FRANÇA,QUINZINHO,NI,CHEFE/LÍDER,NI,NI,NI,NI
3,4,JOSÉ ERIVALDO DA PAZ,ÍNDIO,NI,CHEFE/LÍDER,NI,NI,NI,NI
4,5,LUCAS MANOEL DA SILVA,PROJETO,CV,SOLDADO DO TRÁFICO,NI,NI,NI,CLIMA BOM/FRONTEIRA


## Facções


In [43]:
print(df.groupby("faccao").size())

faccao
CV         75
NEUTRO     11
NI        107
PCC        33
SF          2
dtype: int64


In [53]:
df["faccao_id"] = df.faccao.copy()

In [54]:
df.faccao_id.replace(
    {
        "CV": 1,
        "PCC": 2,
        "NEUTRO": 3,
        "NI": 4,
        "SF": 5,
    },
    regex=True,
    inplace=True,
)

In [69]:
df.head(5)

,Unnamed: 0,nome_completo,vulgo_alcunha,faccao,faccao_funcao,bairro_atual,cidade_atual,uf_atual,area_atuacao
0,1,KAYO NASCIMENTO DE MAGALHÃES,CABRA/ 99,NI,FINANCEIRO,NI,NI,NI,NI
1,2,JOSÉ MARIANO DOS SANTOS,RATO,NI,BRAÇO DIREITO,NI,NI,NI,NI
2,3,GIVALDO BARBOSA DE FRANÇA,QUINZINHO,NI,CHEFE/LÍDER,NI,NI,NI,NI
3,4,JOSÉ ERIVALDO DA PAZ,ÍNDIO,NI,CHEFE/LÍDER,NI,NI,NI,NI
4,5,LUCAS MANOEL DA SILVA,PROJETO,CV,SOLDADO DO TRÁFICO,NI,NI,NI,CLIMA BOM/FRONTEIRA


## Função na Facção


In [70]:
print(df.groupby("faccao_funcao").size())

faccao_funcao
BRAÇO ARMADO               1
BRAÇO DIREITO              4
CHEFE/LÍDER               56
DISCIPLINA                 1
DISTRIBUIDOR/LOGÍSTICA    12
FINANCEIRO                 3
GERENTE                   37
NI                        12
OUTROS                     1
PAIOL/GUARDA-ROUPAS        4
SOLDADO DO TRÁFICO        74
VENDEDOR                  23
dtype: int64


In [71]:
df["funcao_id"] = df.faccao_funcao.copy()

In [72]:
df.funcao_id.replace(
    {
        "CHEFE/LÍDER": 1,
        "BRAÇO DIREITO": 2,
        "BRAÇO ARMADO": 3,
        "GERENTE": 4,
        "FINANCEIRO": 5,
        "DISTRIBUIDOR/LOGÍSTICA": 6,
        "VENDEDOR": 7,
        "PAIOL/GUARDA-ROUPAS": 8,
        "SOLDADO DO TRÁFICO": 9,
        "OUTROS": 10,
    },
    regex=True,
    inplace=True,
)

In [73]:
df.head(15)

,Unnamed: 0,nome_completo,vulgo_alcunha,faccao,faccao_funcao,bairro_atual,cidade_atual,uf_atual,area_atuacao,funcao_id
0,1,KAYO NASCIMENTO DE MAGALHÃES,CABRA/ 99,NI,FINANCEIRO,NI,NI,NI,NI,5
1,2,JOSÉ MARIANO DOS SANTOS,RATO,NI,BRAÇO DIREITO,NI,NI,NI,NI,2
2,3,GIVALDO BARBOSA DE FRANÇA,QUINZINHO,NI,CHEFE/LÍDER,NI,NI,NI,NI,1
3,4,JOSÉ ERIVALDO DA PAZ,ÍNDIO,NI,CHEFE/LÍDER,NI,NI,NI,NI,1
4,5,LUCAS MANOEL DA SILVA,PROJETO,CV,SOLDADO DO TRÁFICO,NI,NI,NI,CLIMA BOM/FRONTEIRA,9
5,6,VICTOR EMANUEL OLIVEIRA DOS SANTOS,VULGO DA CRUZ,NI,SOLDADO DO TRÁFICO,NI,NI,NI,"CLIMA BOM, ROSANE COLLOR",9
6,7,GILVANIO MARCOS OLIVEIRA SANTOS,GIL,NI,GERENTE,NI,NI,NI,GROTA DO CIGANO/ PORTO DE PEDRAS,4
7,8,DERIVALDO DA SILVA SANTOS,DERI,NI,CHEFE/LÍDER,NI,NI,NI,GROTA DO CIGANO,1
8,9,DARLAN DARLAN PARCEIRA,SCOOBY,PCC,GERENTE,NI,NI,NI,MORRO DO ARI,4
9,10,NI,NI,NI,NI,NI,NI,NI,NI,NI


## Bairros


In [76]:
df.bairro_atual = df.bairro_atual.str.upper()

In [77]:
print(df.groupby("bairro_atual").size())

bairro_atual
ALTO DA BARRA               1
ALTO DE SÃO MARCOS          1
BARRA NOVA                  3
BARRO DURO                  1
BENEDITO BENTES             4
BENEDITO BENTES             2
CABREIRAS                   3
CENTRO                      3
CIDADE UNIVERSITÁRIA        2
COMPLEXO DO ALEMÃO          1
CONJUNTO BARRO PRETO 2      1
DENISSON AMORIM             1
DOM CONSTANTINO             4
ERICK FERRAZ                2
JD BRASÍLIA                 1
JOSÉ DIAS                   1
JOSÉ NUNES                  1
MANOEL TELES                3
MASSAGUEIRA                 1
MASSARANDUBA                1
NI                        174
PARAISÓPOLIS                1
PEDRAS                      2
POEIRA                      1
PONTA GROSSA                1
POÇO                        2
QUILOMBO                    1
SANTA ISABEL                1
SANTA LUCIA                 1
SEM BAIRRO ATUAL            1
TABULEIRO                   1
TAPERAGUÁ                   3
VERGEL DO LAGO.            

In [99]:
df_grouped = df.groupby("bairro_atual").size().reset_index(name="Contagem")

In [100]:
df_grouped.to_csv("bairros.csv", index=True)

## Cidade


In [78]:
df.cidade_atual = df.cidade_atual.str.upper()

In [97]:
print(df.groupby("cidade_atual").size())

cidade_atual
ARAPIRACA                6
BARRA DE SÃO MIGUEL      3
CIDADE DE ITAJAÍ         1
CORURIPE                 5
GIRAU DO PONCIANO        1
JOAQUIM GOMES            1
MACEIÓ                  14
MACEIÓ                   1
MARECHAL DEODORO        22
NI                     166
PENEDO                   5
RIO DE JANEIRO           1
SÃO PAULO                2
dtype: int64


In [95]:
df.cidade_atual.replace(
    {
        "ARAPIRACA - AL": "ARAPIRACA",
        "CORURIPE-AL": "CORURIPE",
        "MARECHALDEODORO": "MARECHAL DEODORO",
        "PENEDO - AL": "PENEDO",
        "SAO PAULO": "SÃO PAULO",
        "MACEIO": "MACEIÓ",
    },
    regex=True,
    inplace=True,
)

## UF atual


In [98]:
print(df.groupby("uf_atual").size())

uf_atual
AL     56
NI    167
PE      1
RJ      1
SC      1
SP      2
dtype: int64


##


In [ ]:
# nomes = df[["nome_completo", "vulgo_alcunha", "cpf", "rg", "cnh"]].copy()
# fotos = df[["foto_recente", "fotos_tatuagens"]].copy()
# faccao = df[["faccao"]].copy()
# funcao = df[["faccao_funcao"]].copy()
# area_atuacao = df[["area_atuacao"]].copy()
# ja_atuou = df[["area_que_ja_atuou"]].copy()
# filiacao = df[["filiacao", "data_nascimento"]].copy()
# alvo_sspal = df[["alvo_sspal", "qual_operacao_foi_alvo"]].copy()
# reeducando = df[["registro_como_reeducando", "período_como_reeducando"]].copy()
# conjugue = df[["nome_completo_conjugue", "tempo_conjugue", "uf_atual"]].copy()
# endereco_anterior = df[["endereco_anterior"]].copy()
# endereco_familiar = df[["endereco_familiar"]].copy()
# endereco_atual = df[
#     ["endereco_atual", "bairro_atual", "cidade_atual", "uf_atual"]
# ].copy()
# bairro = df[["bairro_atual"]].copy()
# cidade = df[["cidade_atual"]].copy()
# uf = df[["uf_atual"]].copy()
# cep = df[["cep"]].copy()
# coordenadas = df[["latitude_atual", "longitude_atual"]].copy()
# analises_vinculos = df[
#     [
#         "nome_completo",
#         "vulgo_alcunha",
#         "faccao",
#         "faccao_funcao",
#         "bairro_atual",
#         "cidade_atual",
#         "uf_atual",
#         "area_atuacao",
#     ]
# ].copy()

In [ ]:
path_faccoes = "./"

In [ ]:
# # df.to_csv(path_faccoes + "faccionados.csv", index=True)
# df.to_csv(path_faccoes + "faccionados_neo4j_00.csv", index=True)

# # csvs desmembrados
# nomes.to_csv(path_faccoes + "nomes.csv", index=True)
# fotos.to_csv(path_faccoes + "fotos.csv", index=True)
# faccao.to_csv(path_faccoes + "faccao.csv", index=True)
# funcao.to_csv(path_faccoes + "funcao.csv", index=True)
# area_atuacao.to_csv(path_faccoes + "area_atuacao.csv", index=True)
# ja_atuou.to_csv(path_faccoes + "ja_atuou.csv", index=True)
# filiacao.to_csv(path_faccoes + "filiacao.csv", index=True)
# alvo_sspal.to_csv(path_faccoes + "alvo_sspal.csv", index=True)
# reeducando.to_csv(path_faccoes + "reeducando.csv", index=True)
# conjugue.to_csv(path_faccoes + "conjugue.csv", index=True)
# endereco_anterior.to_csv(path_faccoes + "endereco_anterior.csv", index=True)
# endereco_familiar.to_csv(path_faccoes + "endereco_familiar.csv", index=True)
# endereco_atual.to_csv(path_faccoes + "endereco_atual.csv", index=True)
# bairro.to_csv(path_faccoes + "bairro.csv", index=True)
# cidade.to_csv(path_faccoes + "cidade.csv", index=True)
# uf.to_csv(path_faccoes + "uf.csv", index=True)
# cep.to_csv(path_faccoes + "cep.csv", index=True)
# coordenadas.to_csv(path_faccoes + "coordenadas.csv", index=True)
# coordenadas.to_csv(path_faccoes + "coordenadas.csv", index=True)
# analises_vinculos.to_csv(path_faccoes + "analises_vinculos.csv", index=True)